In [1]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from tensorflow.keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from tensorflow.keras import backend as Kb
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import add, concatenate
from keras.layers import AveragePooling2D
from keras.utils import plot_model
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
 
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from operator import truediv


from plotly.offline import init_notebook_mode
 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
!pip install spectral
import spectral
import time 

2024-07-12 15:36:03.283785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 15:36:03.311273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 15:36:03.311321: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 15:36:03.330604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 15:36:04.310638: W tensorflow/com


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
def applyFA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    fa = FactorAnalysis(n_components=numComponents, random_state=0)
    newX = fa.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, fa
## GLOBAL VARIABLES
dataset = 'SA'
test_ratio = 0.95
windowSize = 12
def loadData(name):
    data_path = os.path.join(os.getcwd(),'data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    
    return data, labels
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [3]:
X, y = loadData(dataset)

X.shape, y.shape

((512, 217, 204), (512, 217))

In [4]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((54129, 12, 12, 204), (54129,))

In [5]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((2706, 12, 12, 204), (51423, 12, 12, 204), (2706,), (51423,))

In [6]:
Xtrain = Xtrain.transpose(0, 3, 1, 2).reshape(2706, 1, 204, 12, 12)

In [7]:
Xtrain.shape

(2706, 1, 204, 12, 12)

In [8]:
Xtest = Xtest.transpose(0, 3, 1, 2).reshape(51423, 1, 204, 12, 12)

In [9]:
Xtest.shape

(51423, 1, 204, 12, 12)

In [10]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from ptflops import get_model_complexity_info
from transformer import transformer
from embeddings import (PatchEmbeddings, CLSToken, PositionalEmbeddings)

class Pooling(nn.Module):
    def __init__(self, pool: str = "mean"):
        super().__init__()
        if pool not in ["mean", "cls"]:
            raise ValueError("pool must be one of {mean, cls}")

        self.pool_fn = self.mean_pool if pool == "mean" else self.cls_pool

    def mean_pool(self, x: torch.Tensor) -> torch.Tensor:
        return x.mean(dim=1)

    def cls_pool(self, x: torch.Tensor) -> torch.Tensor:
        return x[:, 0]

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.pool_fn(x)

class Classifier(nn.Module):
    def __init__(self, dim: int, num_classes: int):
        super().__init__()
        self.model = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(in_features=dim, out_features=num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

class Res2(nn.Module):
    def __init__(self, in_channels, inter_channels, kernel_size, padding=0):
        super(Res2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, inter_channels, kernel_size=kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm2d(inter_channels)
        self.conv2 = nn.Conv2d(inter_channels, in_channels, kernel_size=kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, X):
        X = F.relu(self.bn1(self.conv1(X)))
        X = self.bn2(self.conv2(X))
        return X

class Res(nn.Module):
    def __init__(self, in_channels, kernel_size, padding, groups_s):
        super(Res, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, padding=padding, groups=groups_s)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, padding=padding, groups=groups_s)
        self.bn2 = nn.BatchNorm2d(in_channels)
        self.res2 = Res2(in_channels, 32, kernel_size=kernel_size, padding=padding)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        Z = self.res2(X)
        return F.relu(X + Y + Z)

class CTMixer(nn.Module):
    def __init__(self, channels, num_classes, image_size, datasetname, num_layers: int=1, num_heads: int=4, 
                 patch_size: int = 1, emb_dim: int = 128, head_dim = 64, hidden_dim: int = 64, pool: str = "mean"):
        super().__init__()
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.channels = channels
        self.image_size = image_size
        self.num_patches = (image_size // patch_size) ** 2
        self.num_patch = int(math.sqrt(self.num_patches))
        self.act = nn.ReLU(inplace=True)
        patch_dim = channels * patch_size ** 2

        # Conv Preprocessing Module (Ref-SPRN)
        if datasetname == 'IndianPines':
            groups = 10  # Adjusted groups to match channels
            groups_width = 40  # Adjusted width to match groups
        elif datasetname == 'PaviaU':
            groups = 5
            groups_width = 64
        elif datasetname == 'Salinas':
            groups = 11
            groups_width = 37
        elif datasetname == 'Houston':
            groups = 5
            groups_width = 64
        else:
            groups = 11
            groups_width = 37

        new_bands = math.ceil(channels/groups) * groups
        patch_dim = (groups*groups_width) * patch_size ** 2
        pad_size = new_bands - channels
        self.pad = nn.ReplicationPad3d((0, 0, 0, 0, 0, pad_size))
        self.conv_1 = nn.Conv2d(new_bands, groups*groups_width, (1, 1), groups=groups)
        self.bn_1 = nn.BatchNorm2d(groups*groups_width)
        self.res0 = Res(groups*groups_width, (3, 3), (1, 1), groups_s=groups)

        # Dual Residual Block (Ref-RDACN)
        self.bn1 = nn.BatchNorm2d(emb_dim)
        self.conv1 = nn.Conv2d(emb_dim, 64, kernel_size=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1, groups=64)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, emb_dim, kernel_size=1, padding=0)

        # Vision Transformer
        self.patch_embeddings = PatchEmbeddings(patch_size=patch_size, patch_dim=patch_dim, emb_dim=emb_dim)
        self.pos_embeddings = PositionalEmbeddings(num_pos=self.num_patches, dim=emb_dim)
        self.transformer = transformer(dim=emb_dim, num_layers=num_layers, num_heads=num_heads, 
                                       head_dim=head_dim, hidden_dim=hidden_dim, num_patch=self.num_patch, patch_size=patch_size)
        self.dropout = nn.Dropout(0.5)

        self.pool = Pooling(pool=pool)
        self.classifier = Classifier(dim=emb_dim, num_classes=num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pad(x).squeeze(1)
        b, c, h, w = x.shape
        x = F.relu(self.bn_1(self.conv_1(x)))
        x = self.res0(x)

        x4 = self.patch_embeddings(x)
        x5 = self.pos_embeddings(x4)
        x6 = self.transformer(x5)

        x4_c = x4.reshape(b, -1, h, w)
        x_c1 = self.conv1(self.act(self.bn1(x4_c)))
        x_c2 = self.conv2(self.act(self.bn2(x_c1)))
        x_c3 = self.conv3(self.act(self.bn3(x_c2)))

        x7 = self.pool(self.dropout(x6 + x_c3.reshape(b, h*w, -1)))

        return self.classifier(x7)


if __name__ == '__main__':
    input = torch.randn(size=(2706, 1, 204, 12, 12))
    print("input shape:", input.shape)
    model = CTMixer(channels=204, num_classes=16, image_size=12, datasetname='PaviaU', num_layers=1, num_heads=4)

    # Get the model summary
    summary(model, input_size=(2706, 1, 204, 12, 12), device="cpu")

    # Calculate the FLOPs
    macs, params = get_model_complexity_info(model, (1, 204, 12, 12), as_strings=True, print_per_layer_stat=True)
    print(f"MACs: {macs}")
    print(f"Parameters: {params}")


input shape: torch.Size([2706, 1, 204, 12, 12])
320
CTMixer(
  650.93 k, 97.246% Params, 93.49 MMac, 99.666% MACs, 
  (act): ReLU(0, 0.000% Params, 36.86 KMac, 0.039% MACs, inplace=True)
  (pad): ReplicationPad3d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, (0, 0, 0, 0, 0, 1))
  (conv_1): Conv2d(13.44 k, 2.008% Params, 1.94 MMac, 2.063% MACs, 205, 320, kernel_size=(1, 1), stride=(1, 1), groups=5)
  (bn_1): BatchNorm2d(640, 0.096% Params, 92.16 KMac, 0.098% MACs, 320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res0): Res(
    555.94 k, 83.055% Params, 80.05 MMac, 85.340% MACs, 
    (conv1): Conv2d(184.64 k, 27.585% Params, 26.59 MMac, 28.343% MACs, 320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=5)
    (bn1): BatchNorm2d(640, 0.096% Params, 92.16 KMac, 0.098% MACs, 320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(184.64 k, 27.585% Params, 26.59 MMac, 28.343% MACs, 320, 320, kernel_size=(3, 3), stride=(1, 1), 

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
import numpy as np
import time

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(Xtrain, dtype=torch.float32)
y_train_tensor = torch.tensor(ytrain, dtype=torch.long)
X_test_tensor = torch.tensor(Xtest, dtype=torch.float32)
y_test_tensor = torch.tensor(ytest, dtype=torch.long)

# Create custom datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [12]:
# Initialize model, loss function, and optimizer
model = CTMixer(channels=204, num_classes=16, image_size=12, datasetname='Salinas', num_layers=1, num_heads=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
model

407


CTMixer(
  (act): ReLU(inplace=True)
  (pad): ReplicationPad3d((0, 0, 0, 0, 0, 5))
  (conv_1): Conv2d(209, 407, kernel_size=(1, 1), stride=(1, 1), groups=11)
  (bn_1): BatchNorm2d(407, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res0): Res(
    (conv1): Conv2d(407, 407, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=11)
    (bn1): BatchNorm2d(407, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(407, 407, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=11)
    (bn2): BatchNorm2d(407, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res2): Res2(
      (conv1): Conv2d(407, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 407, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(407, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [13]:
# Training loop
num_epochs = 50
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')
end_time = time.time()
print(f'Training time: {end_time - start_time:.2f} seconds')

Epoch [1/50], Loss: 0.9738
Epoch [2/50], Loss: 0.6351
Epoch [3/50], Loss: 0.4097
Epoch [4/50], Loss: 0.3627
Epoch [5/50], Loss: 0.2661
Epoch [6/50], Loss: 0.2641
Epoch [7/50], Loss: 0.1852
Epoch [8/50], Loss: 0.1791
Epoch [9/50], Loss: 0.2223
Epoch [10/50], Loss: 0.1294
Epoch [11/50], Loss: 0.1408
Epoch [12/50], Loss: 0.1170
Epoch [13/50], Loss: 0.1611
Epoch [14/50], Loss: 0.1103
Epoch [15/50], Loss: 0.1024
Epoch [16/50], Loss: 0.0892
Epoch [17/50], Loss: 0.1604
Epoch [18/50], Loss: 0.1036
Epoch [19/50], Loss: 0.0645
Epoch [20/50], Loss: 0.0532
Epoch [21/50], Loss: 0.0869
Epoch [22/50], Loss: 0.0945
Epoch [23/50], Loss: 0.0934
Epoch [24/50], Loss: 0.0909
Epoch [25/50], Loss: 0.0783
Epoch [26/50], Loss: 0.0679
Epoch [27/50], Loss: 0.0659
Epoch [28/50], Loss: 0.0576
Epoch [29/50], Loss: 0.0578
Epoch [30/50], Loss: 0.0417
Epoch [31/50], Loss: 0.0818
Epoch [32/50], Loss: 0.0464
Epoch [33/50], Loss: 0.0351
Epoch [34/50], Loss: 0.0532
Epoch [35/50], Loss: 0.0353
Epoch [36/50], Loss: 0.0413
E

In [14]:
# Evaluate the model
model.eval()
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():
    start_time = time.time()
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
    end_time = time.time()  # End timing
# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy of the model on the test set: {100 * accuracy:.2f}%')
print(f'Test time: {end_time - start_time:.2f} seconds')

Accuracy of the model on the test set: 99.03%
Test time: 53.27 seconds


In [15]:
# Classification report
class_report = classification_report(all_labels, all_preds, target_names=[f'Class {i}' for i in range(16)])
print('Classification Report:')
print(class_report)

# Confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print('Confusion Matrix:')
print(conf_matrix)

# Kappa accuracy
kappa = cohen_kappa_score(all_labels, all_preds)
print(f'Kappa Accuracy: {kappa:.4f}')

Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00      1909
     Class 1       1.00      1.00      1.00      3540
     Class 2       0.99      1.00      1.00      1877
     Class 3       1.00      1.00      1.00      1324
     Class 4       0.99      1.00      1.00      2544
     Class 5       1.00      1.00      1.00      3761
     Class 6       1.00      1.00      1.00      3400
     Class 7       0.98      0.98      0.98     10707
     Class 8       1.00      1.00      1.00      5893
     Class 9       1.00      0.99      0.99      3114
    Class 10       1.00      1.00      1.00      1015
    Class 11       1.00      1.00      1.00      1831
    Class 12       0.98      1.00      0.99       870
    Class 13       0.99      0.98      0.99      1016
    Class 14       0.98      0.96      0.97      6905
    Class 15       1.00      1.00      1.00      1717

    accuracy                           0.99     51423
   

In [16]:
# Overall Accuracy
oa = accuracy_score(all_labels, all_preds)

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
# Calculate per-class accuracy from the confusion matrix
class_accuracy = cm.diagonal() / cm.sum(axis=1)
# Average Accuracy
aa = np.mean(class_accuracy)

# Kappa Coefficient
kappa = cohen_kappa_score(all_labels, all_preds)

print(f'Overall Accuracy (OA): {oa:.4f}')
print(f'Average Accuracy (AA): {aa:.4f}')
print(f'Kappa Coefficient: {kappa:.4f}')
for i, acc in enumerate(class_accuracy): print(f'Class {i+1} Accuracy: {acc:.4f}')

Overall Accuracy (OA): 0.9903
Average Accuracy (AA): 0.9941
Kappa Coefficient: 0.9892
Class 1 Accuracy: 1.0000
Class 2 Accuracy: 1.0000
Class 3 Accuracy: 0.9968
Class 4 Accuracy: 0.9985
Class 5 Accuracy: 0.9992
Class 6 Accuracy: 0.9997
Class 7 Accuracy: 0.9991
Class 8 Accuracy: 0.9842
Class 9 Accuracy: 0.9990
Class 10 Accuracy: 0.9897
Class 11 Accuracy: 0.9990
Class 12 Accuracy: 1.0000
Class 13 Accuracy: 0.9977
Class 14 Accuracy: 0.9783
Class 15 Accuracy: 0.9635
Class 16 Accuracy: 1.0000
